In [16]:
# importeer alle benodigde libraries
import xml.etree.ElementTree as ET
import requests
import json

In [17]:
# importeer alle nodige scripts
from Scripts.Create_Medewerkers import extract_all_medewerkers
from Scripts.Social_Media_Functions import update_social_media, update_social_media_organisaties, double_checker, double_checker2, update_org_files
from Scripts.Create_Orgs import create_organisations

In [18]:
# Download de recentste versie van het export00 bestand
xml_url = 'https://organisaties.overheid.nl/archive/exportOO.xml'

# Download de XML file
response = requests.get(xml_url)
xml_content = response.content

# Save de XML File
with open('Files/XML/exportOO_full.xml', 'wb') as file:
    file.write(xml_content)

# Parse de XML file
tree = ET.parse('Files/XML/exportOO_full.xml')
root = tree.getroot()
schema = str(root.tag.split('}')[0][1:])

# Bepaal de namespaces
namespaces = {
    "p":schema
}

In [19]:
orgs = create_organisations(tree, namespaces)

Organisaties = ['Waterschap', ('Ministerie', False), 'Agentschap', 'Rechtspraak', 'Hoog College van Staat','Gemeente']

for Organisatie in Organisaties:
    
    print('')
    if len(Organisatie) != 2:
        print(f"Verzamelen medewerkers van: {Organisatie}")
        dl = extract_all_medewerkers(Organisatie, tree, namespaces, orgs)
    else:
        print(f"Verzamelen medewerkers van: {Organisatie[0]}")
        dl = extract_all_medewerkers(Organisatie[0], tree, namespaces, orgs, multi_layer = Organisatie[1])


Newest Version: 2437
Older Version: 2442

Verzamelen medewerkers van: Waterschap
aantal medewerkers in XML: 48
aantal medewerkers extracted in JSON: 48

Verzamelen medewerkers van: Ministerie
aantal medewerkers in XML: 737
aantal medewerkers extracted in JSON: 56

Verzamelen medewerkers van: Agentschap
aantal medewerkers in XML: 68
aantal medewerkers extracted in JSON: 66

Verzamelen medewerkers van: Rechtspraak
aantal medewerkers in XML: 69
aantal medewerkers extracted in JSON: 69

Verzamelen medewerkers van: Hoog College van Staat
aantal medewerkers in XML: 319
aantal medewerkers extracted in JSON: 319

Verzamelen medewerkers van: Gemeente
aantal medewerkers in XML: 14155
aantal medewerkers extracted in JSON: 14155

resultaten verwijderen fractievoorzitters uit de raadsleden:
Before: 14155
After: 11329
Less items: 2826


Runtime: 13.7 seconde. Wordt volledig veroorzaakt door het downloaden van de ttl files!

In [20]:
# Updaten van de sociale media, wikipedia en Woo-pagina van de organisaties.

x = update_social_media_organisaties()

In [21]:
# Updaten van de sociale media, wikipedia, Voornaam en WorkplaceHomepage van de medewerkers.
Organisaties = ['Waterschap', 'Ministerie', 'Agentschap', 'Rechtspraak', 'Hoog College van Staat', 'Gemeente']

for O in Organisaties:
    y = update_social_media(O)
    print(double_checker(O))
    print(double_checker2(O))
    print(update_org_files(O))
    print('')

0 names changed
0 names changed
Update Waterschap compleet

0 names changed
0 names changed
Update Ministerie compleet

0 names changed
0 names changed
Update Agentschap compleet

0 names changed
0 names changed
Update Rechtspraak compleet

0 names changed
0 names changed
Update Hoog College van Staat compleet

0 names changed
0 names changed
Update Gemeente compleet



In [22]:
counter = 0
with open(f"Files/Json/Medewerkers/Social Media/Social_Media_Gemeente_Raw.json", "r") as file:
            social_media_dict = json.load(file)
            for k,v in social_media_dict.items():
                if v['foaf_firstName'] != '':
                    counter+=1
                    
print(counter)

6830


In [25]:
import os
import pandas as pd

def statistics(Organisatie):
    
    if (os.path.exists(f"Files/Json/Medewerkers/Personen/{Organisatie}.json")) == False:
        return None
    
    with open(f"Files/Json/Medewerkers/Personen/{Organisatie}.json", "r") as file:
        json_data = json.load(file)
        
    
    identifiers = list(json_data[0]['infobox']['foi_dossiers']['0'].keys())
    id_dict = {item: 0 for item in identifiers}
    
    for dct in json_data:
        for k,v in dct['infobox']['foi_dossiers'].items():
            for ID in identifiers:
                if v[ID] != '':
                    id_dict[ID] += 1
                        
                        
                        
    Max = id_dict['dc_identifier']
    Keep = ['foaf_firstName', 'foi_linkedin', 'foi_twitter', 'foi_wikipedia', 'foaf_workplaceHomepage']
            
    filtered_dict = {'Total': Max}

    for k, v in id_dict.items():
        if k in Keep:
            filtered_dict[k] = v

    return filtered_dict


columns = ['Totaal', 'Voornaam', 'Linkedin', 'Twitter', 'Wikipedia', 'Website']

df = pd.DataFrame(columns=['Totaal', 'Voornaam', 'Linkedin', 'Twitter', 'Wikipedia', 'Website'])

for o in Organisaties:
    stats = statistics(o).values()
    
    column_name = o
    values = stats
    df.loc[column_name] = values
    
df.loc['Total'] = df.sum()
display(df)


,Totaal,Voornaam,Linkedin,Twitter,Wikipedia,Website
Waterschap,48,36,37,5,15,39
Ministerie,56,54,45,14,34,0
Agentschap,66,53,47,7,4,6
Rechtspraak,69,44,41,0,7,5
Hoog College van Staat,319,229,141,37,164,15
Gemeente,11329,6833,5976,396,578,3233
Total,11887,7249,6287,459,802,3298


In [26]:
def org_stats():
    if (os.path.exists(f"Files/Json/Organisaties/alle_organisaties.json")) == False:
        return None
    
    with open(f"Files/Json/Organisaties/alle_organisaties.json", "r") as file:
        json_data = json.load(file)
        
    
    identifiers = list(json_data['infobox']['foi_dossiers']['0'].keys())
    id_dict = {item: 0 for item in identifiers}
    
    for k,v in json_data['infobox']['foi_dossiers'].items():
        for ID in identifiers:
            if v[ID] != '':
                if v['dc_publisher'] != '':
                    id_dict[ID] += 1
                        
                        
                        
    Max = id_dict['dc_identifier']
    Keep = ['foi_linkedin', 'foi_twitter', 'foi_wikipedia', 'foi_Woo_URL']
            
    filtered_dict = {'Total': Max}

    for k, v in id_dict.items():
        if k in Keep:
            filtered_dict[k] = v

    df = pd.DataFrame(columns=['Totaal', 'Linkedin', 'Twitter', 'Wikipedia', 'Woo-Pagina'])
    df.loc['Organisaties'] = filtered_dict.values()
    
    return df


display(org_stats())

,Totaal,Linkedin,Twitter,Wikipedia,Woo-Pagina
Organisaties,1044,594,468,486,395


In [27]:
import json

In [28]:
with open(f"Files/Json/Organisaties/alle_organisaties.json", "r") as file:
        json_data = json.load(file)
        

new_dict = dict()
count = 0
for k, v in json_data['infobox']['foi_dossiers'].items():
    if v['dc_publisher'] != '':
        new_dict[count] = v
        count+=1
        
final_dict = {'infobox': {'foi_totalDossiers': count, 'foi_dossiers': new_dict}}

Json = json.dumps(final_dict, indent=4)

with open(f"Files/Json/Organisaties/alle_TOOI_organisaties.json", "w") as file:
    file.write(Json)